In [ ]:
import pandas as pd
from collections import Counter
import requests as r
import matplotlib.pyplot as plt
import heapq
import numpy as np
%matplotlib inline

In [ ]:
url = 'https://gist.githubusercontent.com/ctindall/4588786/raw/d3c331af8a4af0b3a36b3d27afc99042f1733038/us-constitution.txt'
#url = 'https://raw.githubusercontent.com/grantaguinaldo/ssie/master/ssie500/hw2/pride_prejudice.txt?token=AE6SX6CH5KNONKHOSPX44OK7UOHBO'
text = r.get(url).text.strip().replace('\n', '')
textDict = dict(Counter(text))

df = pd.DataFrame({'char': list(textDict.keys()), 'char_count': list(textDict.values())})
df['freq'] = df['char_count'] * (1/df.char_count.sum())
df = df.sort_values(by='freq', ascending=False).reset_index(drop=True)
freq = {row['char']:row['freq'] for index, row in df.iterrows()}

In [ ]:
#https://stackoverflow.com/questions/46825980/huffman-coding-tree-traversal
#https://gist.github.com/nboubakr/0eec4ea650eeb6dc21f9
#https://docs.python.org/3/library/heapq.html#module-heapq
heap_freq = []
for char, frq in freq.items():
    heap_freq.append([frq, [char, ""]])

 #Sorts in decending order, in place and places data in the heap structure.
heapfy = heapq.heapify(heap_freq)

#Loop through heap elements as it gets shorter.
while len(heap_freq) > 1:
    # Take the two smallest values from  the heap.
    left_node = heapq.heappop(heap_freq)
    right_node = heapq.heappop(heap_freq)
    
    # Return the list elements from left node.
    left_elements = left_node[1:]
    # Return the list elements from  the right node.
    right_elements = right_node[1:]
    
    # Scan each element in the left node and append a zero.
    for each in left_elements:
        each[1] = each[1] + '0'
      
    # Scan each element in the right node and append a one.    
    for each in right_elements:
        each[1] = each[1] + '1'

    # Append the resulting elements (coded char and new freq) back into the heap
    heapq.heappush(heap_freq, [left_node[0] + right_node[0]] + left_node[1:] + right_node[1:])

# Create dict of all of the coded char
huffDict = {each[0]:each[1] for each in heapq.heappop(heap_freq)[1:]}
huffDict

In [ ]:
df['entropy'] = -1 * df['freq'] * np.log2(df['freq'])
info_entropy = sum(df.entropy.to_list())
print('Information Entropy: {:.3f} bits'.format(info_entropy))

In [ ]:
encoded = ''
for i in range(len(text)):
    encoded += huffDict[text[i]]
    
print('Average Codeword Length After Encoding: {:.3f} bits'.format(len(encoded)/len(text)))

In [ ]:
plt.figure(figsize=(20, 10))
plt.bar(df.char, df.freq, color='blue', alpha=0.5)
plt.yticks(fontsize=14)
plt.xticks(fontsize=14)
plt.xlabel('Character', fontsize=16)
plt.ylabel('Character Frequency', fontsize=16)
plt.title('Character Frequency Chart of the US Constitution', fontsize=19)
plt.show()